In [21]:
#import dependency
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
from datetime import datetime
import os
import pandas as pd

In [23]:
# Output File (CSV)
output_data_file = "Source Data/Cafes_and_restaurants__with_seating_capacity.csv"

In [24]:
#reading the files
raw_df = pd.read_csv(output_data_file)
raw_df.head()

,Census year,Block ID,Property ID,Base property ID,Building address,CLUE small area,Business address,Industry (ANZSIC4) code,Industry (ANZSIC4) description,Seating type,Number of seats,x coordinate,y coordinate,Location
0,2002,2,111467,103973,0 King Street MELBOURNE 3000,Melbourne (CBD),469-479 King Street MELBOURNE 3000,8921,Zoological and Botanical Gardens Operation,Seats - Indoor,113,144.9595,-37.8206,POINT (144.9595 -37.8206)
1,2002,2,111467,103973,0 King Street MELBOURNE 3000,Melbourne (CBD),469-479 King Street MELBOURNE 3000,8921,Zoological and Botanical Gardens Operation,Seats - Outdoor,38,144.9595,-37.8206,POINT (144.9595 -37.8206)
2,2002,4,103972,103972,363-397 Flinders Street MELBOURNE 3000,Melbourne (CBD),"Vault 12, 387 Flinders Street MELBOURNE 3000",9539,Other Personal Services n.e.c.,Seats - Indoor,15,144.9623,-37.8195,POINT (144.9623 -37.8195)
3,2002,4,103972,103972,363-397 Flinders Street MELBOURNE 3000,Melbourne (CBD),"Vault 1, 363-367 Flinders Street MELBOURNE 3000",9111,Health and Fitness Centres and Gymnasia Operation,Seats - Indoor,20,144.9623,-37.8195,POINT (144.9623 -37.8195)
4,2002,4,103972,103972,363-397 Flinders Street MELBOURNE 3000,Melbourne (CBD),"Vault 1, 363-367 Flinders Street MELBOURNE 3000",9111,Health and Fitness Centres and Gymnasia Operation,Seats - Outdoor,18,144.9623,-37.8195,POINT (144.9623 -37.8195)


In [31]:
#see the variety of industry within the dataset
unique_industry = raw_df['Industry (ANZSIC4) description'].unique()
industry_count = len(unique_industry)
print(unique_industry)
print(f'Number of total industry: {industry_count}')

['Zoological and Botanical Gardens Operation'
 'Other Personal Services n.e.c.'
 'Health and Fitness Centres and Gymnasia Operation'
 'Other Gambling Activities' 'Cafes and Restaurants'
 'Takeaway Food Services' 'Pubs, Taverns and Bars' 'Accommodation'
 'Internet Service Providers and Web Search Portals'
 'Fruit and Vegetable Retailing' 'Other Specialised Food Retailing'
 'Clubs (Hospitality)' 'Oil and Gas Extraction'
 'Other Interest Group Services n.e.c.'
 'Business and Professional Association Services'
 'Sport and Camping Equipment Retailing' 'Newspaper and Book Retailing'
 'Convenience Store' 'Professional Photographic Services'
 'Motor Cycle Retailing' 'Higher Education'
 'Fruit and Vegetable Wholesaling' 'Other Administrative Services n.e.c.'
 'Fresh Meat, Fish and Poultry Retailing'
 'Computer System Design and Related Services' 'Common Area'
 'Performing Arts Venue Operation'
 'Sports and Physical Recreation Clubs and Sports Professionals'
 'Fuel Retailing'
 'Horse and Dog Rac

In [38]:
#see the total unique address (by the business address) within the dataset
unique_business_address = raw_df['Business address'].unique()
unique_business_address_count = len(unique_business_address)
print(unique_business_address)
print(f'Number of total unique address: {unique_business_address_count}')

['469-479 King Street MELBOURNE 3000'
 'Vault 12, 387 Flinders Street MELBOURNE 3000'
 'Vault 1, 363-367 Flinders Street MELBOURNE 3000' ...
 '29 Melrose Street NORTH MELBOURNE VIC 3051'
 '31 Melrose Street NORTH MELBOURNE VIC 3051'
 '17 Melrose Street NORTH MELBOURNE VIC 3051']
Number of total unique address: 6917


In [39]:
#see the total unique location within the dataset, which should match with the business address
unique_Location = raw_df['Location'].unique()
unique_Location_count = len(unique_business_address)
print(unique_Location)
print(f'Number of total unique Location: {unique_Location_count}')

['POINT (144.9595 -37.8206)' 'POINT (144.9623 -37.8195)'
 'POINT (144.9653 -37.8187)' ... 'POINT (144.9353015 -37.79142168)'
 'POINT (144.9356959 -37.79223169)' 'POINT (144.9311131 -37.79436219)']
Number of total unique Location: 6917


In [40]:
by_building = raw_df.set_index('Building address').groupby(['Building address'])
by_building.head()

,Census year,Block ID,Property ID,Base property ID,CLUE small area,Business address,Industry (ANZSIC4) code,Industry (ANZSIC4) description,Seating type,Number of seats,x coordinate,y coordinate,Location
Building address,,,,,,,,,,,,,
0 King Street MELBOURNE 3000,2002,2,111467,103973,Melbourne (CBD),469-479 King Street MELBOURNE 3000,8921,Zoological and Botanical Gardens Operation,Seats - Indoor,113,144.95950,-37.82060,POINT (144.9595 -37.8206)
0 King Street MELBOURNE 3000,2002,2,111467,103973,Melbourne (CBD),469-479 King Street MELBOURNE 3000,8921,Zoological and Botanical Gardens Operation,Seats - Outdoor,38,144.95950,-37.82060,POINT (144.9595 -37.8206)
363-397 Flinders Street MELBOURNE 3000,2002,4,103972,103972,Melbourne (CBD),"Vault 12, 387 Flinders Street MELBOURNE 3000",9539,Other Personal Services n.e.c.,Seats - Indoor,15,144.96230,-37.81950,POINT (144.9623 -37.8195)
363-397 Flinders Street MELBOURNE 3000,2002,4,103972,103972,Melbourne (CBD),"Vault 1, 363-367 Flinders Street MELBOURNE 3000",9111,Health and Fitness Centres and Gymnasia Operation,Seats - Indoor,20,144.96230,-37.81950,POINT (144.9623 -37.8195)
363-397 Flinders Street MELBOURNE 3000,2002,4,103972,103972,Melbourne (CBD),"Vault 1, 363-367 Flinders Street MELBOURNE 3000",9111,Health and Fitness Centres and Gymnasia Operation,Seats - Outdoor,18,144.96230,-37.81950,POINT (144.9623 -37.8195)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72 Stubbs Street KENSINGTON VIC 3031,2020,2539,614669,614669,Kensington,72 Stubbs Street KENSINGTON VIC 3031,4511,Cafes and Restaurants,Seats - Indoor,30,144.93530,-37.79142,POINT (144.9353015 -37.79142168)
25-77 Stubbs Street KENSINGTON VIC 3031,2020,2540,617955,617955,Kensington,61 Stubbs Street KENSINGTON VIC 3031,4511,Cafes and Restaurants,Seats - Outdoor,8,144.93570,-37.79223,POINT (144.9356959 -37.79223169)
25-77 Stubbs Street KENSINGTON VIC 3031,2020,2540,617955,617955,Kensington,61 Stubbs Street KENSINGTON VIC 3031,4511,Cafes and Restaurants,Seats - Indoor,24,144.93570,-37.79223,POINT (144.9356959 -37.79223169)
